In [ ]:
# Based on https://github.com/weiaicunzai/pytorch-cifar100

# Models

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler, LambdaLR
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
def _weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

## Baseline Module

In [12]:
class PreActBasic(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride):
        super().__init__()

        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels*PreActBasic.expansion, kernel_size=3, padding=1, bias=False)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * PreActBasic.expansion:
            self.shortcut = nn.Conv2d(in_channels, out_channels * PreActBasic.expansion, 1, stride=stride, bias=False)

    def forward(self, x):
        res = self.residual(x)
        shortcut = self.shortcut(x)

        return res + shortcut

In [13]:
class PreActBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride):
        super().__init__()

        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * PreActBottleneck.expansion, 1, bias=False)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * PreActBottleneck.expansion:
            self.shortcut = nn.Conv2d(in_channels, out_channels * PreActBottleneck.expansion, 1, stride=stride, bias=False)

    def forward(self, x):
        res = self.residual(x)
        shortcut = self.shortcut(x)

        return res + shortcut

## ReLU Dropped Module

In [14]:
class ReLUDroppedBasic(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride):
        super().__init__()

        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels*PreActBasic.expansion, kernel_size=3, padding=1, bias=False)
        )

        self.reludrop = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels*PreActBasic.expansion, kernel_size=3, padding=1, bias=False)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * PreActBasic.expansion:
            self.shortcut = nn.Conv2d(in_channels, out_channels * PreActBasic.expansion, 1, stride=stride, bias=False)

    def forward(self, x):
        res = self.residual(x)
        shortcut = self.shortcut(x)
        reludrop = self.shortcut(x)

        return res + shortcut + reludrop

In [15]:
class ReLUDroppedBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride):
        super().__init__()

        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * PreActBottleneck.expansion, 1, bias=False)
        )

        self.reludrop = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * PreActBottleneck.expansion, 1, bias=False)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * PreActBottleneck.expansion:
            self.shortcut = nn.Conv2d(in_channels, out_channels * PreActBottleneck.expansion, 1, stride=stride, bias=False)

    def forward(self, x):
        res = self.residual(x)
        shortcut = self.shortcut(x)
        reludrop = self.reludrop(x)

        return res + shortcut + reludrop

## Network Definition

In [16]:
class PreActResNet(nn.Module):
    def __init__(self, block, num_block, class_num=100):
        super().__init__()
        self.input_channels = 64

        self.pre = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.stage1 = self._make_layers(block, num_block[0], 64, 1)
        self.stage2 = self._make_layers(block, num_block[1], 128, 2)
        self.stage3 = self._make_layers(block, num_block[2], 256, 2)
        self.stage4 = self._make_layers(block, num_block[3], 512, 2)

        self.linear = nn.Linear(self.input_channels, class_num)

    def _make_layers(self, block, block_num, out_channels, stride):
        layers = []

        layers.append(block(self.input_channels, out_channels, stride))
        self.input_channels = out_channels * block.expansion

        while block_num - 1:
            layers.append(block(self.input_channels, out_channels, 1))
            self.input_channels = out_channels * block.expansion
            block_num -= 1

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.pre

        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)

        x = F.adaptive_avg_pool2d(x, 1)
        x = x.view(x.size(0), -1)
        x = self.linear(x)

        return x

In [57]:
def basicresnet70(num_class=100):
    return PreActResNet(PreActBasic, [3, 4, 24, 3], num_class)

def bottleneckresnet101(num_class=100):
    return PreActResNet(PreActBottleneck, [3, 4, 23, 3], num_class)

In [62]:
def basicresnet34_reludrop(num_class=100):
    return PreActResNet(ReLUDroppedBasic, [3, 4, 6, 3], num_class)

def bottleneckresnet50_reludrop(num_class=100):
    return PreActResNet(ReLUDroppedBottleneck, [4, 4, 5, 3], num_class)

In [63]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model_1 = basicresnet70()
model_2 = bottleneckresnet101()
model_3 = basicresnet34_reludrop()
model_4 = bottleneckresnet50_reludrop()
print(count_parameters(model_1))
print(count_parameters(model_2))
print(count_parameters(model_3))
print(count_parameters(model_4))

42577700
42685732
42426020
42317988


In [55]:
def basicresnet70_test(num_class=10):
    return PreActResNet(PreActBasic, [3, 4, 24, 3], num_class)

def bottleneckresnet101_test(num_class=10):
    return PreActResNet(PreActBottleneck, [3, 4, 23, 3], num_class)

def basicresnet34_reludrop_test(num_class=10):
    return PreActResNet(ReLUDroppedBasic, [3, 4, 6, 3], num_class)

def bottleneckresnet50_reludrop_test(num_class=10):
    return PreActResNet(ReLUDroppedBottleneck, [4, 4, 5, 3], num_class)

In [56]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model_1 = basicresnet70_test()
model_2 = bottleneckresnet101_test()
model_3 = basicresnet34_reludrop_test()
model_4 = bottleneckresnet50_reludrop_test()
print(count_parameters(model_1))
print(count_parameters(model_2))
print(count_parameters(model_3))
print(count_parameters(model_4))

42531530
42501322
42379850
42133578


# Utils

In [ ]:
def get_train_dataloader(mean, std, batch_size=128, num_workers=4, shuffle=True):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    cifar100_train = torchvision.datasets.CIFAR100(
        root='./data', train=True, download=True, transform=transform_train)
    cifar100_train_loader = DataLoader(
        cifar100_train, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)

    return cifar100_train_loader

def get_test_dataloader(mean, std, batch_size=128, num_workers=4, shuffle=False):
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    cifar100_test = torchvision.datasets.CIFAR100(
        root='./data', train=False, download=True, transform=transform_test)
    cifar100_test_loader = DataLoader(
        cifar100_test, shuffle=shuffle, num_workers = num_workers, batch_size=batch_size)

    return cifar100_test_loader

In [ ]:
# https://github.com/Armour/pytorch-nn-practice/blob/master/utils/meanstd.py
def compute_mean_std(num_class=100):
    mean = [0.50707516, 0.48654887, 0.44091784]
    std = [0.26733429, 0.25643846, 0.27615047]

    if num_class == 10:
        mean = [0.49139968, 0.48215841, 0.44653091]
        std = [0.24703223, 0.24348513, 0.26158784]

    return mean, std

In [ ]:
# def func(epoch):
#     if epoch < 40:
#         return 0.5
#     elif epoch < 70:
#         return 0.5 ** 2
#     elif epoch < 90:
#         return 0.5 ** 3
#     else:
#         return 0.5 ** 4

# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = LambdaLR(optimizer, lr_lambda = func)